Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import joblib
#import shap
import matplotlib.pylab as pl

Reading the data

In [2]:
train_data = pd.read_csv("./Data/cleaned_train.csv")
test_data = pd.read_csv("./Data/cleaned_test.csv")
target = pd.read_csv("./Data/cleaned_target.csv")

target.set_index("Id", inplace = True)

Label encoding the features

In [3]:
enc = LabelEncoder()
train_result = {}
test_result = {}
for col in train_data.columns:
    if (train_data[col].dtype == "int" or train_data[col].dtype == "float"):
        train_result[col] = train_data[col]
        test_result[col] = test_data[col]
    else:
        train_result[col] = pd.Series(enc.fit_transform(train_data[col]))
        test_result[col] = pd.Series(enc.transform(test_data[col]))
train_features = pd.DataFrame(train_result)
test_features = pd.DataFrame(test_result)

train_features.set_index("Id", inplace = True)
test_features.set_index("Id", inplace = True)

In [4]:
train_data.columns

Index(['Id', 'LotArea', 'OverallQual', 'GarageCars', 'TotRmsAbvGrd',
       'OverallCond', 'GrLivArea', 'TotalBsmtSF', 'Fireplaces', 'CentralAir',
       'Street', 'YrSold', 'KitchenQual', 'ExterQual', 'ExterCond',
       'HeatingQC', 'GarageQual', 'PoolQC', 'BsmtCond', 'Fence',
       'BsmtExposure', 'BldgType', 'Functional', 'ProxPos', 'ProxRoad',
       'ProxRail', 'Heating', 'BasementQualFactor', 'FullBath', 'HalfBath',
       'PorchSF', 'Neighborhood', 'MSSubClass', 'SaleCondition'],
      dtype='object')

Random Forest

In [ ]:
rf = RandomForestRegressor(n_jobs = -1, n_estimators = 140)
rf_params = {'max_features': range(3,10), 'max_depth': range(1, 13), 'min_samples_leaf': [4, 7, 14, 70, 140]}
gs_rf = GridSearchCV(rf, rf_params, cv = 10, return_train_score = True)

In [ ]:
np.random.seed(0)
%time gs_rf.fit(train_features, target['SalePrice'])

In [ ]:
gs_rf.best_score_

In [ ]:
gs_rf.best_params_

In [ ]:
importances = pd.Series(gs_rf.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
importances[:20]

In [ ]:
model_rf = gs_rf.best_estimator_

XGBoost with early stopping

In [ ]:
train_features_grid, train_features_validation, target_grid, target_validation = train_test_split(train_features, target, test_size = 0.2)

In [ ]:
xgb = XGBRegressor(n_jobs = -1, n_estimators = 1000)
xgb_params = {'max_depth': range(1, 6), 'eta': np.linspace(0.0001, 0.5, num = 50, endpoint = True)}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [ ]:
np.random.seed(0)
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

XGBoost without early stopping

In [5]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree")
eta = np.linspace(0.0001, 0.5, num = 10, endpoint = True)
n_estimators = [25 * i for i in range(1,11)]
xgb_params = {'max_depth': range(2, 6), 'eta': eta, 'n_estimators' : n_estimators}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [ ]:
np.random.seed(0)
%time gs_xgb.fit(train_features, target, verbose = 0)

XGBoost outcomes

In [ ]:
gs_xgb.best_score_

In [ ]:
gs_xgb.best_params_

In [ ]:
importances = pd.Series(gs_xgb.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
importances[:20]

In [ ]:
model_xgb = gs_xgb.best_estimator_

Running the model on the test dataset

In [ ]:
pred_xgb = pd.Series(model_xgb.predict(test_features))
prediction_xgb = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_xgb})
prediction_xgb.set_index("Id", inplace = True)

In [ ]:
prediction_xgb.to_csv("./Data/prediction_xgb.csv")

In [ ]:
pred_rf = pd.Series(model_rf.predict(test_features))
prediction_rf = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_rf})
prediction_rf.set_index("Id", inplace = True)

In [ ]:
prediction_rf.to_csv("./Data/prediction_rf.csv")

In [ ]:
pred_av = (pred_xgb + pred_rf) / 2
prediction_av = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_av})
prediction_av.set_index("Id", inplace = True)

In [ ]:
prediction_av.to_csv("./Data/prediction_av.csv")

Save encoder and model for later

In [ ]:
# save the model to disk
filename = 'fitted_model.sav'
joblib.dump(model, filename)
filename = 'fitted_encoder.sav'
joblib.dump(enc, filename)

Checking model on the test data set